# Pedestrian street networks for 21 cities
Using OpenStreetMap as a source for 
* complete road network, and 
* a pedestrian 'walk/cycle'network. 
* calculating intersection density for the pedestrian network

The process iterates over a list of city names, whose 10km buffered boundary shape files have been pre-prepared.

Carl Higgs
1 October 2018

## OSM set up

In [2]:
# Libraries used for OSM conversion
import os
import sys
import subprocess as sp
from datetime import datetime

# Libraries used for OSMnx analyses and output
import networkx as nx
import osmnx as ox
import requests
import fiona
ox.config(use_cache=True, log_console=True)
ox.__version__

from shapely.geometry import shape, MultiPolygon, Polygon

# define pedestrian network custom filter (based on OSMnx 'walk' network type, without the cycling exclusion)
pedestrian = (
             '["area"!~"yes"]' 
             '["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]'
             '["foot"!~"no"]'  
             '["service"!~"private"]' 
             '["access"!~"private"]'
             )

## Extract OSM using 10km buffered study region .poly files

The idea with this step is to extract OSM for each buffered study region, and then this file would be used to build the all non-private roads, and pedestrian roads networks.  However there is apparently a bug with the OSMnx 'graph_from_file' command; as such, OSMnx networks were constructed using the OSM version current on Overpass.de at date of processing (2/10/2018).

In [17]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of source OSM file
osm_dir = 'D:/osm/planet_archives/planet-latest_20181001.osm.pbf'

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/'

# conversion settings
exe = 'osmconvert64-0.8.8p.exe'
exepath = 'D:/osm/'
osm_format = 'osm'

# output suffix
suffix = '_20181001'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           if os.path.isfile('{}'.format(studyregion)):
             print('.osm file {} already exists'.format(studyregion))
            
           if not os.path.isfile('{}'.format(studyregion)):
             command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
             sp.call(command, shell=True, cwd=exepath)
             count+=1
             print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                  (datetime.now() - subtime).total_seconds()/60))
            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

.osm file D:/ntnl_li_2018_template/data/study_region/adelaide/adelaide_gccsa_2016_10000m_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/albury_wodonga/AlburyWodonga_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/ballarat/Ballarat_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/bendigo/Bendigo_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/bris/bris_gccsa_2016_10000m_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/cairns/Cairns_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/canberra/canberra_gccsa_2016_10000m_20181001.osm already exists
 Extraction of .osm file for darwin_gccsa_2016_10000m complete in 19.9 minutes.
.osm file D:/ntnl_li_2018_template/data/study_region/geelong/Geelong_20181001.osm already exists
.osm file D:/ntnl_li_2018_template/data/study_region/goldcoast_tweedheads/GoldC

In [ ]:
# Extract Australia
# location of source OSM file
osm_dir = 'D:/osm/planet_archives/planet-latest_20181001.osm.pbf'

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/australia'

# conversion settings
exe = 'osmconvert_0.8.10_largepoly_6000004_x64.exe'
exepath = 'D:/osm/osmctools/src/'
osm_format = 'osm'

# output suffix
suffix = '_20181001'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           if os.path.isfile('{}'.format(studyregion)):
             print('.osm file {} already exists'.format(studyregion))
            
           if not os.path.isfile('{}'.format(studyregion)):
             command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
             print(command)
             sp.call(command, shell=True, cwd=exepath)
             count+=1
             print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                        (datetime.now() - subtime).total_seconds()/60))

            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

osmconvert_0.8.10_largepoly_6000004_x64.exe D:/osm/planet_archives/planet-latest_20181001.osm.pbf -B=D:/ntnl_li_2018_template/data/study_region/australia\Australia.poly -o=D:/ntnl_li_2018_template/data/study_region/australia/Australia_20181001.osm


Note that the '21 cities' directory has since been re-set up as 'study_regions'. 

## Get networks and save as graphs (retain_all = True)

In [16]:
# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/'
# output suffix
suffix = '_20181001'
count = 0
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if "10kmBuff" in file: 
           os.rename(os.path.join(root,file),os.path.join(root,file.replace('10kmBuff','_sua_2018_10000m_epsg4326').lower()))

for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('2016','2018')
           if os.path.isfile(os.path.join(root,
                   'osm_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,suffix = suffix))):
             print('Pedestrian road network for {} has already been processed'.format(studyregion))
           else:
             subtime = datetime.now()
             # Extract pedestrian network
             c = fiona.open(fullfile)   
             polygon = shape(next(iter(c))['geometry'])
             # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
             W = ox.graph_from_polygon(polygon,  network_type= 'all', retain_all = True)
             ox.save_graphml(W, filename=os.path.join(root,
                                                           'osm_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                       suffix = suffix)), 
                             folder=None, gephi=False)
             ox.save_graph_shapefile(W, 
                                     filename=os.path.join(root,
                                                           'osm_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                       suffix = suffix)))
             W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian, retain_all = True)
             ox.save_graphml(W, filename=os.path.join(root,
                                                           'osm_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                       suffix = suffix)), 
                             folder=None, gephi=False)
             ox.save_graph_shapefile(W, 
                                     filename=os.path.join(root,
                                                           'osm_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                       suffix = suffix)))  
             print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                    (datetime.now() - subtime).total_seconds()/60))         
        count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))        

Saved graph object and shapefile for adelaide_gccsa_2018_10000m_epsg4326 in 23.0 minutes.
Saved graph object and shapefile for alburywodonga__sua_2018_10000m_epsg4326 in 2.0 minutes.
Saved graph object and shapefile for ballarat__sua_2018_10000m_epsg4326 in 2.6 minutes.
Saved graph object and shapefile for bendigo__sua_2018_10000m_epsg4326 in 1.7 minutes.
Saved graph object and shapefile for bris_gccsa_2018_10000m_epsg4326 in 24.9 minutes.
Saved graph object and shapefile for cairns__sua_2018_10000m_epsg4326 in 1.5 minutes.
Saved graph object and shapefile for canberra_gccsa_2018_10000m_epsg4326 in 11.0 minutes.
Saved graph object and shapefile for geelong__sua_2018_10000m_epsg4326 in 4.2 minutes.
Saved graph object and shapefile for goldcoast__sua_2018_10000m_epsg4326 in 8.8 minutes.
Saved graph object and shapefile for hobart_gccsa_2018_10000m_epsg4326 in 3.1 minutes.
Saved graph object and shapefile for launceston__sua_2018_10000m_epsg4326 in 1.4 minutes.
Saved graph object and shap

** Note that the below analyses are tangential and were performed prior to the above **

## Clean intersections for 21 cities

In [20]:
# import geopandas as gp
from sqlalchemy import *
from geoalchemy2 import Geometry
import subprocess as sp
# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region'
# output suffix
suffix = '_20181001'
stub = '_pedestrian{suffix}.graphml'.format(suffix = suffix)
db = 'li_intersections_2018'
user = 'python'
pwd = '***REMOVED***'
engine = create_engine('postgresql://{}:{}@localhost:5432/{}'.format(user,pwd,db))
conn = engine.connect()
places = ['"{}"'.format(x) for x in engine.table_names() if x not in ['spatial_ref_sys','intersections']]
count = 0
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(stub): 
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.split('_')[1]
           if '"{}"'.format(studyregion) not in places and studyregion!='10km':
             G = ox.load_graphml(fullfile)
             G_proj = ox.project_graph(G)
             intersections = ox.clean_intersections(G_proj, tolerance=15, dead_ends=False)
             intersections.crs = G_proj.graph['crs']
             intersections_latlon = intersections.to_crs(epsg=4326)
             # to sql  - works well!

             statement = '''
               DROP TABLE IF EXISTS {studyregion};
               CREATE TABLE {studyregion} (point_4326 geometry);
               INSERT INTO {studyregion} (point_4326) VALUES {points};
               ALTER TABLE {studyregion} ADD COLUMN geom geometry;
               UPDATE {studyregion} SET geom = ST_Transform(point_4326,7845);
               ALTER TABLE {studyregion} DROP COLUMN point_4326;
             '''.format(studyregion = studyregion,
                        points = ', '.join(["(ST_GeometryFromText('{}',4326))".format(x.wkt) for x in intersections_latlon]))  
             conn.execute(statement)   
             places.append('"{}"'.format(studyregion))
           
        count+=1          
## output clean intersections for all study regions to geopackage                 
command = (
          'ogr2ogr -overwrite -f GPKG {dir}/{db}.gpkg '
          ' PG:"host=localhost user={user} dbname={db} password={pwd}" '
          ' {places}'.format(dir = search_dir,
                             db = db,
                             user = user,
                             pwd = pwd,
                             places = ' '.join(places))
           )
sp.call(command)
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))     

Elapsed time was 5775.9 minutes


In [24]:
# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region'
# output suffix
suffix = '_20181001'
db = 'li_intersections_2018'
user = 'python'
pwd = '***REMOVED***'
engine = create_engine('postgresql://{}:{}@localhost:5432/{}'.format(user,pwd,db))
conn = engine.connect()
places = ['"{}"'.format(x) for x in engine.table_names() if x not in ['spatial_ref_sys','intersections']]
command = (
          'ogr2ogr -overwrite -f GPKG {dir}/{db}.gpkg '
          ' PG:"host=localhost user={user} dbname={db} password={pwd}" '
          ' {places}'.format(dir = search_dir,
                             db = db,
                             user = user,
                             pwd = pwd,
                             places = ' '.join(places))
           )
print("The following command may not launch from within the Jupyter notebook, but try running it at the command prompt -- seems to work from there!\n\n{}".format(command))
sp.call(command)

The following command may not launch from within the Jupyter notebook, but try running it at the command prompt -- seems to work from there!

ogr2ogr -overwrite -f GPKG D:/ntnl_li_2018_template/data/study_region/li_intersections_2018.gpkg  PG:"host=localhost user=python dbname=li_intersections_2018 password=***REMOVED***"  "darwin" "geelong" "canberra" "hobart" "goldcoast" "launceston" "mackay" "cairns" "melb" "mitchell" "newcastle" "townsville" "perth" "sunshinecoast" "syd" "toowoomba" "westernsydney" "wollongong" "adelaide" "alburywodonga" "ballarat" "bendigo" "bris"


0

## Extract legacy Brisbane OSM for comparison of Open Space areas

In [27]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of source OSM file
osm_dir = 'D:/osm/planet_archives/planet-180514.osm.pbf'
### NOTE! This doesn't work --- the download was corrupted....

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/bris'

# conversion settings
exe = 'osmconvert64-0.8.8p.exe'
exepath = 'D:/osm/'
osm_format = 'osm'

# output suffix
suffix = '_20181001'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           if os.path.isfile('{}'.format(studyregion)):
             print('.osm file {} already exists'.format(studyregion))
            
           if not os.path.isfile('{}'.format(studyregion)):
             command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
             sp.call(command, shell=True, cwd=exepath)
             count+=1
             print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                  (datetime.now() - subtime).total_seconds()/60))
            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

 Extraction of .osm file for bris_gccsa_2016_10000m complete in 0.7 minutes.

Extracted (or attempted to extract) 1 OSM portions.
Elapsed time was 0.7 minutes


## Test brisbane with retain_all = 'True'

Does this fix the island issue (Stradbroke networks missing) --- answer = yes.

In [7]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/bris'

# output suffix
suffix = '_20181011_retain_all'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('10000m_epsg4326','10km')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all',retain_all ='True')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian,retain_all ='True')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))  

Saved graph object and shapefile for bris_gccsa_2016_10km in 24.8 minutes.
Elapsed time was 24.8 minutes


## Test brisbane with clean_periphery = 'False'
Does this fix the island issue (Stradbroke networks missing) --- No.

In [8]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/bris'

# output suffix
suffix = '_20181011_no_clean_periphery'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('10000m_epsg4326','10km')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all',clean_periphery ='False')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian,clean_periphery ='False')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))  

Saved graph object and shapefile for bris_gccsa_2016_10km in 25.2 minutes.
Elapsed time was 25.2 minutes
